# Tile mosaic image acquisitions from Opera Phenix 

In [1]:
import os
from glob import glob
from tifffile import imread, imwrite
from skimage.transform import resize
import numpy as np
from tqdm.auto import tqdm

# Set your source and output directory
src_dir = '/mnt/OPERA2/Nathan/macrohet_syno/data/PS0000/acquisition/Images'
out_dir = '/home/dayn/analysis/macrohet/data/untiled_images/ds'
scale_factor = 1 / 5.04

os.makedirs(out_dir, exist_ok=True)

# Get all TIFF files in the directory
tiff_paths = sorted(glob(os.path.join(src_dir, "*.tiff")))

for path in tqdm(tiff_paths):
    img = imread(path)
    
    # Resize with anti-aliasing for downscaling
    downscaled = resize(
        img,
        output_shape=(int(img.shape[0] * scale_factor), int(img.shape[1] * scale_factor)),
        order=1,  # bilinear
        preserve_range=True,
        anti_aliasing=True
    ).astype(img.dtype)

    # Save to output directory with same name
    out_path = os.path.join(out_dir, os.path.basename(path))
    imwrite(out_path, downscaled)


PermissionError: [Errno 13] Permission denied: '/mnt/OPERA2/Nathan/macrohet_syno/data/PS0000/acquisition/Images/r03c05f01p01-ch1sk10fk1fl1.tiff'